In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'numpy'

In [6]:
df=pd.read_csv('real_estate_delhi_ncr_arranged.csv')

In [8]:
df

,City,SubLocation,Name,Price,Rate_per_sqft,Bedroom,Status,Transaction,Carpet_area_sqft,Total_area
0,Delhi,AG1 Pocket Vikaspuri,"2 BHK Flat for Sale in AG1 Pocket Vikaspuri, V...",12500000,13228.0,2,Ready to Move,New Property,NaN,944.965225
1,Delhi,Aastha Kunj Apartments,"3 BHK Flat for Sale in Aastha Kunj Apartments,...",13000000,7449.0,3,Ready to Move,New Property,2600.0,1745.200698
2,Delhi,Adarsh Apartment,"2 BHK Flat for Sale in Adarsh Apartment, Rohin...",17000000,6222.0,2,Ready to Move,Resale,2620.0,2732.240437
3,Delhi,Adarsh Nagar Extension,"2 BHK Flat for Sale in Adarsh Nagar Extension,...",10500000,11667.0,2,Ready to Move,Resale,525.0,899.974286
4,Delhi,Aditi Apartment,"3 BHK Flat for Sale in Aditi Apartment, Indrap...",17500000,10606.0,3,Ready to Move,Resale,NaN,1650.009429
...,...,...,...,...,...,...,...,...,...,...
2340,Faridabad,Tirka Colony,"3 BHK House for Sale in Tirka Colony, Mathura ...",4800000,10000.0,3,Ready to Move,Resale,NaN,480.000000
2341,Faridabad,Urban One,"2 BHK Flat for Sale in Urban One, Dayal Bagh C...",3000000,6422.0,2,Ready to Move,Resale,750.0,467.144192
2342,Faridabad,Woodbury Tower,"3 BHK Flat for Sale in Woodbury Tower, Suraj K...",12500000,7645.0,3,Ready to Move,Resale,1150.0,1635.055592
2343,Faridabad,sai park Apartment sector 87,2 BHK Flat for Sale in sai park Apartment sect...,4600000,4875.0,2,Ready to Move,Resale,NaN,943.589744


In [10]:
df.duplicated().sum()

0

In [12]:
df.isnull().sum().sum()

1452

In [14]:
df.isnull().sum()

City                   0
SubLocation            0
Name                   0
Price                  0
Rate_per_sqft         88
Bedroom                0
Status               128
Transaction            1
Carpet_area_sqft    1147
Total_area            88
dtype: int64

In [24]:
# Dropping the carpet_are_sqft
df.drop('Carpet_area_sqft',axis=1,inplace=True)

In [26]:
df.Transaction.value_counts()

Transaction
Resale          1873
New Property     467
Other              4
Name: count, dtype: int64

In [28]:
df=df[df.Transaction=='Resale']

In [30]:
df.isnull().sum()

City               0
SubLocation        0
Name               0
Price              0
Rate_per_sqft     72
Bedroom            0
Status           102
Transaction        0
Total_area        72
dtype: int64

In [32]:
df.to_csv('Resale.csv')

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
import pickle
import warnings
warnings.filterwarnings("ignore")

print("🚀 Starting model training...")

# Load dataset and clean column names
df = pd.read_csv("real_estate_delhi_ncr_arranged.csv")
df.columns = df.columns.str.strip().str.lower()

print(f"📊 Dataset shape: {df.shape}")
print(f"📋 Columns: {list(df.columns)}")

# Drop missing targets
df.dropna(subset=["price"], inplace=True)
df.drop(columns=["unnamed: 0"], errors='ignore', inplace=True)

print(f"📊 After cleaning: {df.shape}")

# Log transform target
df["price"] = np.log1p(df["price"])

# Feature Engineering
if "bhk" in df.columns and "total_area" in df.columns:
    df["bhk_per_1000sqft"] = df["bhk"] / (df["total_area"] / 1000 + 1)
    df["price_per_bhk"] = np.expm1(df["price"]) / (df["bhk"] + 1)

# Remove outliers
numeric_df = df.select_dtypes(include=np.number)
z_scores = np.abs((numeric_df - numeric_df.mean()) / numeric_df.std())
df = df[(z_scores < 3).all(axis=1)]

print(f"📊 After outlier removal: {df.shape}")

# Features and target
X = df.drop(columns=["price"])
y = df["price"]

# Handle skewed numerical features
skewed_cols = ["total_area", "rate_per_sqft", "carpet_area_sqft"]
for col in skewed_cols:
    if col in X.columns:
        X[col] = np.log1p(X[col])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify column types
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"🔢 Numerical columns: {num_cols}")
print(f"📝 Categorical columns: {cat_cols}")

# Preprocessing
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

# Transform features
print("🔄 Transforming features...")
X_train_t = preprocessor.fit_transform(X_train)
X_test_t = preprocessor.transform(X_test)

print(f"✅ Transformed feature shape: {X_train_t.shape}")

# Base models - Fixed XGBoost parameters
xgb = XGBRegressor(
    n_estimators=1000, 
    learning_rate=0.01, 
    max_depth=4,
    subsample=0.7, 
    colsample_bytree=0.7,
    reg_alpha=0.5, 
    reg_lambda=1.0,
    random_state=42, 
    objective='reg:squarederror',
    tree_method='hist',
    device='cpu'  # Fixed: using device instead of predictor
)

rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)

# Stacking Regressor
stack_model = StackingRegressor(
    estimators=[('xgb', xgb), ('rf', rf)],
    final_estimator=Ridge(alpha=1.0),
    n_jobs=-1
)

# Fit model
print("🏋️ Training stacked model...")
stack_model.fit(X_train_t, y_train)

# Predict
print("🔮 Making predictions...")
y_train_pred = np.expm1(stack_model.predict(X_train_t))
y_test_pred = np.expm1(stack_model.predict(X_test_t))

# Evaluate
def evaluate(y_true, y_pred, label):
    y_true_exp = np.expm1(y_true)
    print(f"\n🔍 {label}")
    print(f"R2 Score: {r2_score(y_true_exp, y_pred):.4f}")
    print(f"RMSE: ₹{np.sqrt(mean_squared_error(y_true_exp, y_pred)):,.2f}")
    print(f"MAE: ₹{mean_absolute_error(y_true_exp, y_pred):,.2f}")

evaluate(y_train, y_train_pred, "Train Set - Stacked Model")
evaluate(y_test, y_test_pred, "Test Set - Stacked Model")

# Save the complete pipeline
print("\n💾 Saving model and preprocessor...")

# Save the model
with open('model1.pkl', 'wb') as f:
    pickle.dump(stack_model, f)

# Save the preprocessor
with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

# Save feature information
feature_info = {
    'num_cols': num_cols,
    'cat_cols': cat_cols,
    'feature_names': list(X.columns),
    'target_transformed': True,  # Target was log-transformed
    'skewed_cols_transformed': skewed_cols
}

with open('feature_info.pkl', 'wb') as f:
    pickle.dump(feature_info, f)

# Create a simple model for API testing (fallback)
print("\n🎯 Creating simple model for API...")

# Create a simple dataset for API testing
api_data = pd.DataFrame({
    'purchase_price': [5000000, 3000000, 8000000, 4500000, 6000000],
    'monthly_rent': [35000, 25000, 55000, 32000, 42000],
    'renovation_cost': [200000, 150000, 300000, 180000, 250000]
})

# Create realistic resale values
api_data['resale_value'] = (
    api_data['purchase_price'] * 1.4 +  # 40% appreciation over 5 years
    api_data['renovation_cost'] * 0.6 +  # 60% of renovation cost retained
    api_data['monthly_rent'] * 12 * 2   # 2 years of rent as bonus
)

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

X_simple = api_data[['purchase_price', 'monthly_rent', 'renovation_cost']]
y_simple = api_data['resale_value']

scaler_simple = StandardScaler()
X_simple_scaled = scaler_simple.fit_transform(X_simple)

model_simple = LinearRegression()
model_simple.fit(X_simple_scaled, y_simple)

# Wrapper class for simple model
class SimpleRealEstateModel:
    def __init__(self, model, scaler):
        self.model = model
        self.scaler = scaler
    
    def predict(self, X):
        if isinstance(X, list):
            X = np.array(X).reshape(1, -1)
        elif len(X.shape) == 1:
            X = X.reshape(1, -1)
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled)

simple_model = SimpleRealEstateModel(model_simple, scaler_simple)

# Save simple model as backup
with open('simple_model.pkl', 'wb') as f:
    pickle.dump(simple_model, f)

print("✅ All models saved successfully!")
print("\nFiles created:")
print("- model1.pkl (Stacked model)")
print("- preprocessor.pkl (Data preprocessor)")
print("- feature_info.pkl (Feature metadata)")
print("- simple_model.pkl (Simple model for API testing)")

# Test the simple model
test_input = np.array([[5000000, 40000, 200000]])
prediction = simple_model.predict(test_input)
print(f"\n🧪 Simple model test: ₹{prediction[0]:,.2f}")

ModuleNotFoundError: No module named 'pandas'